# Restaurant Simulation

Goal of this project: to develop a system that simulate keeps track of a restaurant's status for every certain time period (in this case, 20 minutes)

The system will keep track of the following statuses at the restaurant:

    Diners who are 1) seated 
                   2) ordering
                   3) eating
                   4) paying
                   5) leaving
                   
The system will show the restaurant's menu for each diner and allow them to choose whichever menu they want. Based on each diner's menu, the system will also aumatically calculate the cost of the meal before the diner leaves. 

I created 5 different classes (MenuItem, Menu, Diner, Waiter, RestaurantHelper) for the project.

* MenuItem Class & Menu Class: read menu csv file and show four different categories of menu items for a diner to choose
* Diner Class: represents each diner at the restaurant and keeps track of their status and order
* Waiter Class: represents the restaurant's waiter who is in charge of a list of the diners at the restaurant. The waiter takes care of different status of each diner (e.g. the waiter seats a new diner, takes orders if needed, and gives diners their bill)
* RestaurantHelper Class: randomly generates new diners for a restaurant simulation.

In [3]:
import pandas as pd
import random
from bs4 import BeautifulSoup
import urllib.request
import ssl


##### MenuItem Class #####

class MenuItem(object):
    # inputs: str name, str type, float price, str description
    def __init__(self, name, typeOfItem, price, description):
        self.__name = name
        self.__typeOfItem = typeOfItem
        self.__price = price
        self.__description = description

    # Define getters and setters for each instance attribute:
    # Getters:
    # return: str name
    def getName(self):
        return self.__name

    # return: str type
    def getType(self):
        return self.__typeOfItem

    # return: float price
    def getPrice(self):
        return self.__price

    # return: str description
    def getDescription(self):
        return self.__description

    
    # Setters:
    # input: str newName
    # return: none
    # check if the newName consists of the alphabet
    def setName(self, newName):
        if newName.isalpha() == True:       
            self.__name = newName
        else:
            print("Invalid name")

    # input: str newType
    # return: none
    # check if the newType consists of the alphabet
    def setType(self, newType):
        if newType.isalpha() == True:       
            self.__typeOfItem = newType
        else:
            print("Invalid type")

    # input: float newPrice
    # return: none
    # check if the newPrice is a non-negative float
    def setPrice(self, newPrice):
        if newPrice >= 0:       
            self.__price = newPrice
        else:
            print("Invalid price")

    # input: str description
    # return: none
    # check if the newDescr consists of the alphabet
    def setDescription(self, newDescr):
        if newDescr.isalpha() == True:  
            self.__description = newDescr
        else:
            print("Invalid description")

    # input: none
    # return: str msg
    def __str__(self):
        # convert float to string
        msg = self.__name + " (" + self.__typeOfItem + "): $" + str(self.__price) + "\n\t\t" + self.__description
        return msg
    
    

##### Menu Class #####

class Menu(object):
    # each menu item can be classified into one of the 4 types
    MENU_ITEM_TYPES = ["Drink", "Appetizer", "Entree", "Dessert"]

    # input: str name of the csv file
    def __init__(self, fileName):
        # initialize an instance attribute as an empty dictionary that would contain MenuItem objects
        self.__menuItemDictionary = {}  
        dataList = []  # create an empty list
        inputFile = open(fileName, "r") 

        # read the menu csv file
        for line in inputFile:     
            line = line.strip()
            line.split(",")
            dataList.append(line)   # add each line to the empty list
            # dataList is a list of lists, where each mini-list is a list containing 4 pieces of information about each menu item
        
        # create a MenuItem object from each miniList   
        # the order of the miniList is the same as the order of MenuItem object's input
        for miniList in dataList:   
            miniList = miniList.split(",")
            menuItem = MenuItem(miniList[0], miniList[1], miniList[2], miniList[3])

            # add the new MenuItem object to the menuItemDictionary (its type is the key)
            # check if the type of menu exists in the dictionary
            # if the type of menu already exists in the dictionary, 
                # append a new MenuItem object to the list of objects under the key menuItem.getType()
            if menuItem.getType() in self.__menuItemDictionary:    
                self.__menuItemDictionary[menuItem.getType()].append(menuItem)
            # if the type of menu does not exist, 
                # add a new menuItem object to the dictionary by using menuItem.getType() as a key and by creating a list of menuItem objects for the value
            else:   
                self.__menuItemDictionary[menuItem.getType()] = [menuItem]
        inputFile.close()

    # method: getMenuItem
    # get the correct MenuItem object from the dictionary using its type and index position in the list of items
    # input: str a type of menu(menuType), int index position of a certain menu item
    # return: a MenuItem object from the dictionary
    def getMenuItem(self, menuType, indexPosition):
        menuItem = self.__menuItemDictionary[menuType][indexPosition]   # among the values that has a key MenuType, return an object positioned at the indexPosition
        return menuItem


    # method: printMenuItemsByType
    # print a header with the type of menu items, followed by a numbered list of all the menu items of that type
    # input: str a type of menu(menuType)
    # return: none
    def printMenuItemsByType(self, menuType):
        count = 0
        # check if the menuType is in the dictionary
        if menuType.capitalize() in self.__menuItemDictionary:
            print("\n< " + menuType.capitalize() + " >")        # print a header with the type of menu items
            for item in self.__menuItemDictionary[menuType.capitalize()]:       # for each value(menuitem) in dict[key]
                print(str(count), ")", item)
                count += 1


    # method: getNumMenuItemsByType
    # return the number of MenuItems of the input type
    # input: str a type of menu (menuType)
    # return: int the number of MenuItems of the input type
    def getNumMenuItemsByType(self, menuType):
        return len(self.__menuItemDictionary[menuType.capitalize()])        # return the length of the list of the MenuItem objects under the menuType (key)



##### Diner Class #####

class Diner(object):
    # Assume a diner can have 5 different statuses
    STATUSES = ["seated", "ordering", "eating", "paying", "leaving"]

    # input: str diner's name
    # return: none
    def __init__(self, name):
        self.__name = name
        self.__order = []   # an empty list since the diner has not ordered any menu items yet
        self.__status = 0   # set it to 0 since the diner just seated

    # Getters:
    # return: str diner's name
    def getName(self):
        return self.__name

    # return: list of the MenuItem objects ordered by the diner
    def getOrder(self):
        return self.__order

    # return: int that represents the diner's current dining status
    def getStatus(self):
        return self.__status

    # Setters:
    # input: str newName
    # return: none
    # check if the newName consists of the alphabet
    def setName(self, newName):
        if newName.isalpha() == True:       
            self.__name = newName
        else:
            print("Invalid name")

    # input: list newList
    # return: none
    def setOrder(self, newOrder):
        self.__order = newOrder


    # input: int newStatus
    # return: none
    def setStatus(self, newStatus):
        if newStatus.isdigit() == True:
            self.__status = newStatus
        else:
            print("Invalid integer for the diner's current dining status")

    # method updateStatus
    # increase the diner's status by 1
    # input: none
    # return: none
    def updateStatus(self):
        self.__status += 1

    # method addToOrder
    # adds the menu item to the end of the list of menu items (self.__order)
    # input: a MenuItem object
    # return: none
    def addToOrder(self, menuItem):
        self.__order.append(menuItem)       


    # method printOrder
    # print a message containing all the menu items the diner ordered 
    # input: none
    # return: none
    def printOrder(self):
        for menuItemOrderedObject in self.__order:     
            print("-", menuItemOrderedObject)

    # method calculateMealCost
    # calculate the total cost of menu items the diner ordered
    # input: none
    # return: float total cost
    def calculateMealCost(self):
        totalCost = 0.00   # start with 0
        for menuItemOrderedObject in self.__order:
            totalCost += float(menuItemOrderedObject.getPrice())    
        return totalCost

    # input: none
    # return: str msg
    def __str__(self):
        msg = "Diner " + self.__name + " is currently " + Diner.STATUSES[self.__status] + "."
        return msg

    
    
    
##### Waiter Class #####

class Waiter(object):
    # input: a Menu object
    # return: none
    def __init__(self, menu):
        self.__diners = []      # initialize the list of diners to an empty list
        self.__menu = menu

    # method addDiner
    # add the new Diner object to the waiter's list of diners
    # input: a Diner object
    # return: none
    def addDiner(self, diner):
        self.__diners.append(diner)

    # method getNumDiners
    # get the number of diners the waiter is currently taking care of
    # input: none
    # return: int the number of diners
    def getNumDiners(self):
        return len(self.__diners)       
    
    # method printDinerStatuses
    # print all the diners the waiter is keeping track of, grouped by their statuses
    # input: none
    # return: none
    def printDinerStatuses(self):
        for status in Diner.STATUSES:   # loop through each of the possible statuses a diner can have
            print("Diners who are " + status + ":")
            for diner in self.__diners:         # loop through each of diner object in the list of diners (self.__diners)
                if status == Diner.STATUSES[diner.getStatus()]:     # get the string status of the diner (instead of integer) and see if it's equal to status(string)
                    print("\t" + str(diner))


    # method takeOrders
    # if the diner is ordering, print the menu items for each menu type, then ask the diner to order a menu item by selecting a number
    # add the item to the diner, then print the diner's order
    # input: none
    # return: none
    def takeOrders(self):
        for diner in self.__diners:     # loop through each diner in the list of Diner objects
            if diner.getStatus() == 1:      # 'ordering' is at the index position of 1 in STATUSES
                for menuType in Menu.MENU_ITEM_TYPES:   # loop through the different menu types in MENU_ITEM_TYPES
                    self.__menu.printMenuItemsByType(menuType)     # print the menu items of that type

                    # ask the diner to order a menu item by selecting a number
                    dinerChoice = int(input(diner.getName() + ", please select a " + menuType + " menu item number."))

                    # error checking: if the user put the invalid number, which is any number greater than or equal to the length of the list with key menuType OR any negative number
                    while dinerChoice >= self.__menu.getNumMenuItemsByType(menuType) or dinerChoice < 0:
                        dinerChoice = int(input("Please select a " + menuType + " menu item number again."))

                    food = self.__menu.getMenuItem(menuType, dinerChoice)       # use the dinerChoice as the integer for the indexPosition input in the getMenuItem method
                    diner.addToOrder(food)      # add the item to the order
                print(diner.getName() + " ordered: ")
                diner.printOrder()      # print the diner's order

    # method ringUpDiners
    # check if the diner is paying and if he/she is, calculate the diner's meal cost and print out the message
    # input: none
    # return: none
    def ringUpDiners(self):
        for diner in self.__diners:
            if diner.getStatus() == 3:      # 'paying' is at the index position of 3 in STATUSES
                totalCost = diner.calculateMealCost()           # calculate the diner's meal cost
                print(diner.getName() + ", your meal cost is $" + str(totalCost))       # print out the message

    # method removeDoneDiners
    # check if the diner is leaving, and if he/she is, print a message thanking the diner
    # input: none
    # return: none
    def removeDoneDiners(self):
        # loop through the list of diners backwards because the diner will have a different index number once the other diner gets removed
        for i in range(len(self.__diners)-1, -1, -1):       # start: the length of the list of diners-1, end: -1 (not 0), step: -1 (backwards)
            if self.__diners[i].getStatus() == 4:       # 'leaving' is at the index position of 4 in STATUSES
                print(self.__diners[i].getName() + ", thank you for dining with us! Come again soon!")      # print a message thanking the diner
                self.__diners.remove(self.__diners[i])      # remove the diner from the list

    # method advanceDiners
    # allow the waiter to attend to the diners at their various stages as well as move each diner on to the next stage
    # input: none
    # return: none
    def advanceDiners(self):
        self.printDinerStatuses()
        print()     # a blank line after printing out the diners' statuses to make it look nicer for readers to read
        self.takeOrders()
        self.ringUpDiners()
        self.removeDoneDiners()
        for diner in self.__diners:     # for each diner in the list of diners, update their status
            diner.updateStatus()

         
        
##### Restaurant Helper Class #####        
            
class RestaurantHelper(object):
    URL = "http://random-name-generator.info/"

    # Input: none
    # Return value: a new Diner object with a random name
    #             OR
    #             None (aka no new diner has arrived)
    @staticmethod
    def randomDinerGenerator():
        if random.randint(1, 10) % 3 == 0:
            return Diner(RestaurantHelper.generateRandName())
        else:
            return None

    # Input: none
    # Return: random name
    @staticmethod
    def generateRandName():
        context = ssl._create_unverified_context()
        page = urllib.request.urlopen(RestaurantHelper.URL, context=context)
        soup = BeautifulSoup(page.read(), "html.parser")
        orderedListName = soup.find(class_="nameList")
        return orderedListName.find("li").text.split()[0]


##### Main program for restaurant simulation

In [6]:
import datetime
import time

RESTAURANT_NAME = "J's Plate"
restaurantTime = datetime.datetime(2020, 5, 1, 5, 0)

# Create the menu object
menu = Menu('menu.csv')
    
# create the waiter object
waiter = Waiter(menu)
print("Welcome to " + RESTAURANT_NAME + "!")
print(RESTAURANT_NAME + " is now open for dinner.\n")

for i in range(21):
    print("\n--- It is currently", restaurantTime.strftime("%H:%M PM"), "---")
    restaurantTime += datetime.timedelta(minutes=20)

    potentialDiner = RestaurantHelper.randomDinerGenerator()
    if potentialDiner is not None:
        print("\n" + potentialDiner.getName() + " welcome, please be seated!")  # we have a new diner to add to the waiter's list of diners

        waiter.addDiner(potentialDiner)
    waiter.advanceDiners()
    time.sleep(2)

print("\n~~~ ", RESTAURANT_NAME, "is now closed. ~~~")
# After the restaurant is closed, progress any diners until everyone has left
while waiter.getNumDiners():
    print("\n~~~ It is currently", restaurantTime.strftime("%H:%M PM"), "~~~")
    restaurantTime += datetime.timedelta(minutes=15)
    waiter.advanceDiners()
    time.sleep(2)

print("Goodbye!")



Welcome to J's Plate!
J's Plate is now open for dinner.


--- It is currently 05:00 PM ---

Marion welcome, please be seated!
Diners who are seated:
	Diner Marion is currently seated.
Diners who are ordering:
Diners who are eating:
Diners who are paying:
Diners who are leaving:


--- It is currently 05:20 PM ---
Diners who are seated:
Diners who are ordering:
	Diner Marion is currently ordering.
Diners who are eating:
Diners who are paying:
Diners who are leaving:


< Drink >
0 ) Soda (Drink): $1.50
		Choose from sprite coke or pepsi
1 ) Thai Iced Tea (Drink): $3.00
		Glass of thai iced tea
2 ) Coffee (Drink): $1.50
		Black coffee either hot or cold
Marion, please select a Drink menu item number.3
Please select a Drink menu item number again.4
Please select a Drink menu item number again.1

< Appetizer >
0 ) Fresh Spring Rolls (Appetizer): $3.99
		4 vegetarian rolls wrapped in rice paper either fresh or fried
1 ) Cream Cheese Wantons (Appetizer): $5.99
		Vegetarian friendly fried wanto